# Customer Retention: Online Retail

### Project Goal 

To analyze customer purchasing behavior and measure customer retention using Cohort Analysis.

### Methodology 

The analysis segments customers into monthly cohorts based on their first purchase `CohortMonth` and tracks their subsequent activity over time `CohortIndex`. Key metrics calculated across these cohorts include:

- Customer retention rates.
- Average purchase quantity.

### Dataset Summary

- **Type:** Transactional, multivariate, sequential, time-series  
- **Time period:** 01/12/2010 – 09/12/2011  
- **Domain:** UK-based non-store online retail, mainly selling unique gifts; many customers are wholesalers  
- **Instances:** 541,909 transactions  
- **Features:** 8  
  - `InvoiceNo` (transaction ID)  
  - `StockCode` (product ID)  
  - `Description` (product name)  
  - `Quantity` (units per transaction)  
  - `InvoiceDate` (date & time of transaction)  
  - `UnitPrice` (price per unit, £)  
  - `CustomerID` (customer ID)  
  - `Country` (customer location)  
- **Feature types:** Integer, Real, Categorical, Date  
- **Missing values:** None   
- **Reference:** Chen, Laing, Guo (2012), *Journal of Database Marketing and Customer Strategy Management*

### Insight

- Even though retention declines (fewer returning customers), the customers who stay tend to buy in similar or slightly higher quantities, suggesting loyalty among active buyers.

### Recommendations

- Prioritize first-time customer retention through onboarding, engagement, and targeted follow-ups.
- Encourage repeat purchases with loyalty programs, promotions, or personalized offers.
- Upsell or bundle products to maximize value from returning customers who maintain or increase purchase quantities.
- Segment and target active buyers for tailored marketing based on purchase patterns.


---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

### Load Data

In [ ]:
df_online_retail = pd.read_excel("OnlineRetail.xlsx", index_col=0)

In [ ]:
online = df_online_retail.copy()

In [ ]:
online.head()

---

### Cohort Analysis

#### Customer Segmentation

In [ ]:
# Function that truncates a given date object to a first day of the month.
def get_month(x): return dt.datetime(x.year, x.month, 1)

In [ ]:
online["InvoiceMonth"] = online["InvoiceDate"].apply(get_month)

In [ ]:
grouping = online.groupby("CustomerID")["InvoiceMonth"]

In [ ]:
# Assign the smallest `InvoiceMonth` value to each customer
online["CohortMonth"] = grouping.transform("min")

In [ ]:
online.head()

In [ ]:
# Helper function which will extract integer values 
# of the year, month and day from a datetime() object
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

Calculate the number of months between any transaction and the first transaction for each customer. 

In [ ]:
invoice_year, invoice_month, _ = get_date_int(online, "InvoiceMonth")
cohort_year, cohort_month, _ = get_date_int(online, "CohortMonth")

In [ ]:
years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month

Convert the total difference to months by multiplying the year difference by 12 and adding them together. "+1" in the end: so the first month is marked as 1 instead of 0 for easier interpretation.

In [ ]:
online["CohortIndex"] = years_diff * 12 + months_diff + 1

In [ ]:
online.head()

---

### Count monthly active customers from each cohort

Create a groupby object with `CohortMonth` and `CohortIndex`.

In [ ]:
grouping = online.groupby(["CohortMonth", "CohortIndex"])

Count number of customers in each group by applying pandas `nunique()` function

In [ ]:
cohort_data = grouping["CustomerID"].apply(pd.Series.nunique)

Reset the index and create a pandas pivot with `CohortMonth` in the rows, `CohortIndex` in the columns, and `CustomerID` counts as values.

In [ ]:
cohort_data = cohort_data.reset_index()

In [ ]:
cohort_counts = cohort_data.pivot(index="CohortMonth",
                                 columns="CohortIndex",
                                 values="CustomerID")

In [ ]:
cohort_counts

---

### Cohort Metrics

The retention measures how many customers from each of the cohort have returned in the subsequent months.  
Will calculate the ratio of how many of 1st column customers came back in the subsequent months which is the retention rate.

#### Calculate retention rate

1. Store the first column as `cohort_sizes`

In [ ]:
cohort_sizes = cohort_counts.iloc[:,0]

2. Divide all values in he `cohort_counts` table by `cohort_sizes`. Set the `axis=0` to ensure that we divide along the row axis.

In [ ]:
retention = cohort_counts.divide(cohort_sizes, axis=0)

In [ ]:
retention.round(3) * 100

#### Average Quantity

In [ ]:
grouping = online.groupby(["CohortMonth", "CohortIndex"])

In [ ]:
cohort_data = grouping["Quantity"].mean()

In [ ]:
cohort_data = cohort_data.reset_index()

In [ ]:
average_quantity = cohort_data.pivot(index="CohortMonth",
                                    columns="CohortIndex",
                                    values="Quantity")

In [ ]:
average_quantity.round(1)

#### Unit Price

In [ ]:
grouping = online.groupby(["CohortMonth", "CohortIndex"])

In [ ]:
cohort_data = grouping["UnitPrice"].mean()

In [ ]:
cohort_data = cohort_data.reset_index()

In [ ]:
average_price = cohort_data.pivot(index="CohortMonth",
                                    columns="CohortIndex",
                                    values="UnitPrice")

In [ ]:
average_price.round(2)

---

### Cohort Visualization

In [ ]:
retention.index = retention.index.strftime("%Y-%m-%d")
average_quantity.index = average_quantity.index.strftime("%Y-%m-%d")

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Retention rates")
sns.heatmap(data = retention,
           annot = True,
           fmt = ".0%",
           vmin = 0.0,
           vmax = 0.5,
           cmap = "BuGn")

plt.show()

#### Insights:

- **Retention declines** over time in all cohorts, **rarely exceeding 50%**.
- Most cohorts show **non-monotonic patterns**, with occasional increases after initial drops.
- Early months are critical, while later months see minimal repeat activity.

---

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Average Spend by Monthly Cohorts")
sns.heatmap(data=average_quantity, annot=True, cmap="Blues")
plt.show()

#### Insights:

- Average purchase quantity per customer is fairly stable (8–16 units).
- Some cohorts increase over time, suggesting repeat buyers may order more.
- Fluctuations exist, but no clear downward trend; returning customers often maintain or increase their quantity.
- Early months are generally lower, indicating order size grows after the first purchase.

---